In [4]:
import pandas as pd

data = pd.read_csv('model_data_compress.csv')

In [5]:
projects = data.project.unique()
projectVersionMap = {}
for project in projects:
    projectParts = project.split("_")
    groupName = projectParts[0] + "_" + projectParts[1]
    if groupName not in projectVersionMap:
        projectVersionMap[groupName] = []
    projectVersionMap[groupName].append(project)

projectVersionMap

{'smartshark_ant-ivy': ['smartshark_ant-ivy_1.4.1',
  'smartshark_ant-ivy_2.0.0',
  'smartshark_ant-ivy_2.1.0',
  'smartshark_ant-ivy_2.2.0',
  'smartshark_ant-ivy_2.3.0',
  'smartshark_ant-ivy_2.4.0'],
 'smartshark_archiva': ['smartshark_archiva_1.0',
  'smartshark_archiva_1.1',
  'smartshark_archiva_1.2',
  'smartshark_archiva_1.3',
  'smartshark_archiva_2.0.0',
  'smartshark_archiva_2.1.0',
  'smartshark_archiva_2.2.0'],
 'smartshark_calcite': ['smartshark_calcite_1.0.0',
  'smartshark_calcite_1.1.0',
  'smartshark_calcite_1.10.0',
  'smartshark_calcite_1.11.0',
  'smartshark_calcite_1.12.0',
  'smartshark_calcite_1.13.0',
  'smartshark_calcite_1.14.0',
  'smartshark_calcite_1.15.0',
  'smartshark_calcite_1.2.0',
  'smartshark_calcite_1.3.0',
  'smartshark_calcite_1.4.0',
  'smartshark_calcite_1.5.0',
  'smartshark_calcite_1.6.0',
  'smartshark_calcite_1.7.0',
  'smartshark_calcite_1.8.0',
  'smartshark_calcite_1.9.0'],
 'smartshark_cayenne': ['smartshark_cayenne_3.0.0',
  'smartsha

In [6]:

for key in projectVersionMap:
    subData = data[data['project'].isin(projectVersionMap[key])]
    subData.to_csv('data/' + key +'.csv')
    if key == "smartshark_commons-math":
        subData2 = data[~data['project'].isin(projectVersionMap[key])]
        subData2.to_csv('data/' + key +'_train.csv')
    

In [3]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb

params = {}
num_trees=500
start_at = 'Previous_PDE'
found = False
for key in projectVersionMap:
    if start_at == key:
        found = True    
    if not found:
        continue
    
    print("Build Model " + key)
    train = pd.read_csv('data/' + key + '_train.csv')
    columns = train.drop(['project','bug','Result'], axis=1).columns
    gbm = xgb.train(params, xgb.DMatrix(
    train[columns], train['Result']), num_trees)
    gbm.save_model('model/' + key + '.model')

Build Model Previous_PDE
[06:13:50] WARNING: /tmp/pip-build-tyd51mdl/xgboost/build/temp.linux-x86_64-3.6/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Build Model Previous_SWT
[10:30:01] WARNING: /tmp/pip-build-tyd51mdl/xgboost/build/temp.linux-x86_64-3.6/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Build Model Previous_ZXing
[14:42:06] WARNING: /tmp/pip-build-tyd51mdl/xgboost/build/temp.linux-x86_64-3.6/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Build Model Spring_AMQP
[18:42:22] WARNING: /tmp/pip-build-tyd51mdl/xgboost/build/temp.linux-x86_64-3

In [7]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb

params = {}
num_trees=500

train = pd.read_csv('data/smartshark_train.csv')
columns = train.drop(['project','bug','Result'], axis=1).columns
gbm = xgb.train(params, xgb.DMatrix(
train[columns], train['Result']), num_trees)
gbm.save_model('model/smartshark2.model')

[13:48:40] WARNING: /tmp/pip-build-tyd51mdl/xgboost/build/temp.linux-x86_64-3.6/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [5]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb

gbm = xgb.Booster(model_file='model/smartshark2.model')

In [6]:
gbm.get_score()

{'f12': 2613,
 'f6': 3201,
 'f9': 3857,
 'f8': 2071,
 'f11': 407,
 'f5': 2164,
 'f2': 971,
 'f0': 5846,
 'f3': 3829,
 'f7': 1474,
 'f1': 189}

In [10]:
gbm.get_fscore()

{'f9': 4610,
 'f0': 6388,
 'f8': 2221,
 'f12': 3340,
 'f6': 3743,
 'f2': 1170,
 'f5': 1391,
 'f3': 4097,
 'f1': 120,
 'f7': 1428,
 'f11': 91}